## 1、网页分析

百度图片是Ajax加载出来的，加载时其url地址未变，所以通过分析Ajax来完成爬取

谷歌浏览器打开百度图片官网https://image.baidu.com/

随便输入一个关键字搜索

空白处右键点击检查

<img src="图1.png">

<img src="图2.png">

Network选择XHR，然后不断下拉滚动条

<img src="图3.png">

就可以看到很多type为xhr的条目随便点击一个

<img src="图4.png">

headers中X-Requested_With为XMLHttpRequest则其为Ajax加载内容。下面我们分析以下此内容

<img src="图5.png">

我们复制其url地址新建一个窗口打开

<img src="图6.png">

这是一个json格式的数据 我们通过https://www.json.cn/网站解析此JSON

<img src="图7.png">

可以比较直观的看到此JSON文件的内容，所以到此为止我们找到了ajax加载出来的图片的URL地址

备注：这里还有一个objURL，这里是个加密问题

In [11]:
import re

In [14]:
objURL="ippr_z2C$qAzdH3FAzdH3Fk-ffs_z&e3B17tpwg2_z&e3Bv54AzdH3F7rs5w1fAzdH3Ftpj4AzdH3Fda8b88AzdH3F8mAzdH3Fda8b888m8bcddm_7ystj_z&e3Bpi74k_z&e3B0aa_a_z&e3B3r2"

In [15]:
def decode_url(url):
    """
    对百度加密后的地址进行解码
    :param url:百度加密的url
    :return:解码后的url
    """
    table = {'w': "a", 'k': "b", 'v': "c", '1': "d", 'j': "e", 'u': "f", '2': "g", 'i': "h",
             't': "i", '3': "j", 'h': "k", 's': "l", '4': "m", 'g': "n", '5': "o", 'r': "p",
             'q': "q", '6': "r", 'f': "s", 'p': "t", '7': "u", 'e': "v", 'o': "w", '8': "1",
             'd': "2", 'n': "3", '9': "4", 'c': "5", 'm': "6", '0': "7",
             'b': "8", 'l': "9", 'a': "0", '_z2C$q': ":", "_z&e3B": ".", 'AzdH3F': "/"}
    url = re.sub(r'(?P<value>_z2C\$q|_z\&e3B|AzdH3F+)', lambda matched: table.get(matched.group('value')), url)
    return re.sub(r'(?P<value>[0-9a-w])', lambda matched: table.get(matched.group('value')), url)

In [16]:
decode_url(objURL)

'http://b-ssl.duitang.com/uploads/item/201811/16/20181116185226_uylie.thumb.700_0.jpg'

In [81]:
import requests
import json

In [2]:
headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36'}

In [3]:
url='https://image.baidu.com/search/acjson?tn=resultjson_com&ipn=rj&ct=201326592&is=&fp=result&queryWord=%E8%90%8C%E7%8B%97&cl=2&lm=-1&ie=utf-8&oe=utf-8&adpicid=&st=&z=&ic=&hd=&latest=&copyright=&word=%E8%90%8C%E7%8B%97&s=&se=&tab=&width=&height=&face=&istype=&qc=&nc=&fr=&expermode=&force=&pn=30&rn=30&gsm=1e&1589418458900='

In [4]:
response=requests.get(url,headers=headers)

In [5]:
print(response.status_code)

200


## 一页爬取

In [7]:
print(response.text)

{"queryEnc":"%C3%C8%B9%B7","queryExt":"萌狗","listNum":1129,"displayNum":506911,"gsm":"3c","bdFmtDispNum":"约506,000","bdSearchTime":"","isNeedAsyncRequest":0,"bdIsClustered":"1","data":[{    "adType":"0",    "hasAspData":"0","thumbURL":"https://ss3.bdstatic.com/70cFv8Sh_Q1YnxGkpoWK1HF6hhy/it/u=2839518277,361898006&fm=26&gp=0.jpg","middleURL":"https://ss3.bdstatic.com/70cFv8Sh_Q1YnxGkpoWK1HF6hhy/it/u=2839518277,361898006&fm=26&gp=0.jpg",    "largeTnImageUrl":"",    "hasLarge" :0,    "hoverURL":"https://ss3.bdstatic.com/70cFv8Sh_Q1YnxGkpoWK1HF6hhy/it/u=2839518277,361898006&fm=26&gp=0.jpg",    "pageNum":30,    "objURL":"ippr_z2C$qAzdH3FAzdH3Fk-ffs_z&e3B17tpwg2_z&e3Bv54AzdH3F7rs5w1fAzdH3Ftpj4AzdH3Fda8b88AzdH3F8mAzdH3Fda8b888m8bcddm_7ystj_z&e3Bpi74k_z&e3B0aa_a_z&e3B3r2",    "fromURL":"ippr_z2C$qAzdH3FAzdH3Fooo_z&e3B17tpwg2_z&e3Bv54AzdH3Fks52AzdH3F?t1=8a8l8ca9cm",    "fromURLHost":"www.duitang.com",    "currentIndex":"",    "width":700,    "height":701,    "type":"jpg",    "is_gif":0,    "stra

In [8]:
data=json.loads(response.text)

In [55]:
len(data['data'])#返回去检查，一页30条

31

In [56]:
data['data']

[{'adType': '0',
  'hasAspData': '0',
  'thumbURL': 'https://ss3.bdstatic.com/70cFv8Sh_Q1YnxGkpoWK1HF6hhy/it/u=2839518277,361898006&fm=26&gp=0.jpg',
  'middleURL': 'https://ss3.bdstatic.com/70cFv8Sh_Q1YnxGkpoWK1HF6hhy/it/u=2839518277,361898006&fm=26&gp=0.jpg',
  'largeTnImageUrl': '',
  'hasLarge': 0,
  'hoverURL': 'https://ss3.bdstatic.com/70cFv8Sh_Q1YnxGkpoWK1HF6hhy/it/u=2839518277,361898006&fm=26&gp=0.jpg',
  'pageNum': 30,
  'objURL': 'ippr_z2C$qAzdH3FAzdH3Fk-ffs_z&e3B17tpwg2_z&e3Bv54AzdH3F7rs5w1fAzdH3Ftpj4AzdH3Fda8b88AzdH3F8mAzdH3Fda8b888m8bcddm_7ystj_z&e3Bpi74k_z&e3B0aa_a_z&e3B3r2',
  'fromURL': 'ippr_z2C$qAzdH3FAzdH3Fooo_z&e3B17tpwg2_z&e3Bv54AzdH3Fks52AzdH3F?t1=8a8l8ca9cm',
  'fromURLHost': 'www.duitang.com',
  'currentIndex': '',
  'width': 700,
  'height': 701,
  'type': 'jpg',
  'is_gif': 0,
  'strategyAssessment': '3332907538_33_0_0',
  'filesize': '',
  'bdSrcType': '0',
  'di': '8580',
  'pi': '0',
  'is': '0,0',
  'imgCollectionWord': '',
  'replaceUrl': [{'ObjURL': 'http

In [57]:
url_p=[]

In [58]:
for i in range(0,len(data['data'])-1):
    url_p.append(data['data'][i]['thumbURL'])

## 多页爬取

In [62]:
url_all=[]
for i in range(31):
    url_all.append('https://image.baidu.com/search/acjson?tn=resultjson_com&ipn=rj&ct=201326592&is=&fp=result&queryWord=%E8%90%8C%E7%8B%97&cl=2&lm=-1&ie=utf-8&oe=utf-8&adpicid=&st=&z=&ic=&hd=&latest=&copyright=&word=%E8%90%8C%E7%8B%97&s=&se=&tab=&width=&height=&face=&istype=&qc=&nc=&fr=&expermode=&force=&pn='+str(30*i)+'&rn=30&gsm=1e&1589418458900=')
        
        

In [64]:
url_f=[]

In [65]:
for ul in url_all:
    response=requests.get(ul,headers=headers)
    data=json.loads(response.text)
    for i in range(0,len(data['data'])-1):
        url_f.append(data['data'][i]['thumbURL'])  

In [83]:
len(url_f)

930

In [ ]:
for url in url_f:
    num=0
    path_image = path + "/_%s.png" % num
    num+=1
    request.urlretrieve(url, path_image)
    print("正在下载第%d张" % num)


In [103]:
pic_url=url_f[0]

In [106]:
pic_url

'https://ss1.bdstatic.com/70cFuXSh_Q1YnxGkpoWK1HF6hhy/it/u=379153590,3373487903&fm=26&gp=0.jpg'

In [110]:
pic_t = requests.get(pic_url, timeout=15)
string = '/Users/chengqian/Desktop/mypic/'+'第'+str(i)+ '张.jpg'
with open(string, 'wb') as f:
    f.write(pic_t.content)